#### ***Paper Name:*** Feature Selection based Music Selection using Artificial Intelligence (Submitted to CONF-SEML 2023)

***Authors:*** Murari Prasad, Apoorva Bordoloi, Hem Thumar, Manas Saloi, Deepanshu Joshi

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os

import cv2

from tqdm import tqdm
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
encoder = tf.keras.models.load_model('/content/drive/MyDrive/GTZAN/vgg_enc_final_h5', compile=False)
decoder = tf.keras.models.load_model('/content/drive/MyDrive/GTZAN/vgg_dec_final_h5', compile=False)

In [ ]:
joblib.dump(encoder, '/content/drive/MyDrive/encoder2.pkl')
joblib.dump(decoder, '/content/drive/MyDrive/decoder2.pkl')

In [ ]:
"""  224 x 224 x 3 actual """

data_dir = 'drive/MyDrive/GTZAN/mel_spec_padded_vgg/'

images = []
labels = []
for path,_,files in os.walk(data_dir):
    for name in files:
        temp = cv2.imread(os.path.join(path, name), cv2.IMREAD_UNCHANGED)
        images.append(np.array(cv2.cvtColor(temp, cv2.COLOR_BGRA2BGR)))
        labels.append(name)

In [ ]:
img_arrays = np.array(images)
img_arrays = img_arrays / 255.0

In [ ]:
# encoded_1 = encoder(img_arrays[0][np.newaxis,:,:,:]).numpy()
# encoded_2 = encoder(img_arrays[867][np.newaxis,:,:,:]).numpy()

# vec1 = encoded_1.flatten('F')
# vec2 = encoded_2.flatten('F')

In [ ]:
encodings_t1 = encoder(img_arrays[:250])

In [ ]:
encodings_t1.numpy().reshape(250,7*7*512)[:2].shape

(2, 25088)

In [ ]:
import math 
import torch

def cos_sim(v):
    v_inner = inner_product(v)
    v_size = vec_size(v)
    v_cos = v_inner / torch.mm(v_size, v_size.t())
    return v_cos

def vec_size(v):
    return v.norm(dim=-1, keepdim=True)

def inner_product(v):
    return torch.mm(v, v.t())

def euclidean_dist(v, eps=1e-10):
    v_norm = (v**2).sum(-1, keepdim=True)
    dist = v_norm + v_norm.t() - 2.0 * torch.mm(v, v.t())
    dist = torch.sqrt(torch.abs(dist) + eps)
    return dist

def theta(v, eps=1e-5):
    v_cos = cos_sim(v).clamp(-1. + eps, 1. - eps)
    x = torch.acos(v_cos) + math.radians(10)
    return x

def triangle(v):
    theta_ = theta(v)
    theta_rad = theta_ * math.pi / 180.
    vs = vec_size(v)
    x = (vs.mm(vs.t())) * torch.sin(theta_rad)
    return x / 2.

def magnitude_dif(v):
    vs = vec_size(v)
    return (vs - vs.t()).abs()

def sector(v):
    ed = euclidean_dist(v)
    md = magnitude_dif(v)
    sec = math.pi * torch.pow((ed + md), 2) * theta(v)/360.
    return sec

def ts_ss(v):
    tri = triangle(v)
    sec = sector(v)
    return tri * sec

def ts_ss_(v, eps=1e-30, eps2=1e-30):
    # upper ts_ss seems to work... why this dont know
    # reusable compute
    v_inner = torch.mm(v, v.t())
    vs = v.norm(dim=-1, keepdim=True)
    vs_dot = vs.mm(vs.t())

    # compute triangle(v)
    v_cos = v_inner / vs_dot
    v_cos = v_cos.clamp(-1. + eps2, 1. - eps2)  # clamp to avoid backprop instability
    theta_ = torch.acos(v_cos) + math.radians(10)
    theta_rad = theta_ * math.pi / 180.
    tri = (vs_dot * torch.sin(theta_rad)) / 2.

    # compute sector(v)
    v_norm = (v ** 2).sum(-1, keepdim=True)
    euc_dist = v_norm + v_norm.t() - 2.0 * v_inner
    euc_dist = torch.sqrt(torch.abs(euc_dist) + eps)  # add epsilon to avoid srt(0.)
    magnitude_diff = (vs - vs.t()).abs()
    sec = math.pi * (euc_dist + magnitude_diff) ** 2 * theta_ / 360.

    return tri * sec
    


In [ ]:
# vec1 = [1.0,2.0]
# vec2 = [1.0,2.0]
# vec3 = [3.0,4.0]
# vec4 = [33.0,44.0]
v = torch.tensor(encodings_t1.numpy().reshape(250,7*7*512)[:10].astype('float64'))

# print(euclidean_dist(v))
# print(cos_sim(v))
print(ts_ss(v))
print(ts_ss_(v))
s = ts_ss_(v)[1]

tensor([[1.9265e-11, 1.8012e+06, 1.6890e+06, 6.7719e+05, 4.2794e+06, 2.7000e+06,
         3.6339e+06, 1.6234e+06, 2.6431e+06, 3.1536e+06],
        [1.8012e+06, 3.2636e-11, 1.7803e+06, 1.9051e+06, 5.4809e+06, 2.8207e+06,
         4.5353e+06, 2.9595e+06, 3.3968e+06, 4.1065e+06],
        [1.6890e+06, 1.7803e+06, 3.9557e-11, 1.3967e+06, 3.3579e+06, 2.2362e+06,
         3.9147e+06, 2.5196e+06, 2.2719e+06, 3.7443e+06],
        [6.7719e+05, 1.9051e+06, 1.3967e+06, 2.8532e-11, 4.5805e+06, 1.6569e+06,
         4.0145e+06, 2.0218e+06, 3.0471e+06, 3.6542e+06],
        [4.2794e+06, 5.4809e+06, 3.3579e+06, 4.5805e+06, 1.0050e-10, 4.6348e+06,
         4.9272e+06, 5.9011e+06, 4.0460e+06, 4.9815e+06],
        [2.7000e+06, 2.8207e+06, 2.2362e+06, 1.6569e+06, 4.6348e+06, 5.8069e-11,
         4.4158e+06, 4.2115e+06, 3.7833e+06, 4.2952e+06],
        [3.6339e+06, 4.5353e+06, 3.9147e+06, 4.0145e+06, 4.9272e+06, 4.4158e+06,
         6.3902e-11, 3.7424e+06, 3.2819e+06, 4.3468e+06],
        [1.6234e+06, 2.9595

In [ ]:
s.detach().cpu().numpy().copy()

array([[6.19295296e-12, 1.80122215e+06, 1.68900903e+06],
       [1.80122215e+06, 3.10256282e-31, 1.78025455e+06],
       [1.68900903e+06, 1.78025455e+06, 8.47723352e-12]])